In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/test_parse')

In [ ]:
#!pip install natasha ipymarkup

In [ ]:
!pip install pymorphy2

In [11]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import pandas as pd

In [13]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [68]:
df = pd.read_csv('test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [16]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [14]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word] #and word not in stops
    words = [word.normal_form for word in words]

    return words

In [69]:
df['text_norm'] = df['text'].apply(normalize)

In [70]:
df.head()

,dlg_id,line_n,role,text,text_norm
0,0,0,client,Алло,[алло]
1,0,1,manager,Алло здравствуйте,"[алло, здравствуйте]"
2,0,2,client,Добрый день,"[добрый, день]"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне..."
4,0,4,client,Ага,[ага]


In [62]:
greets = []
byes = []
introduced = []
names = []
orgs = []
daytime = ['день', 'утро', 'вечер', 'ночь']

for i, row in df.iterrows():
  greet = False
  bye = False
  intro = False
  name = 'None'
  org = 'None'

  if row['role'] == 'manager':
    for word_id in range(len(row['text_norm'])):
      word = row['text_norm'][word_id]
      if word_id != (len(row['text_norm'])-1):
        next_word = row['text_norm'][word_id+1]
      else:
        next_word = "None"

      #приветствия и прощания
      if word in ['здравствуйте', 'добрый']: #or (word == 'добрый' and next_word in daytime):
        greet = True
      if word in ['свидание', 'хороший']: #будем считать, что фразы "хорошего дня/вечера", "всего хорошего" тоже считаются за прощание
        bye = True

      #парсим имена и организации
      tags = morph.parse(word)[0].tag
      if "Name" in tags:
        if row['text_norm'][word_id-1] in ['это', 'звать', 'я']:
          intro = True
          name = word
      if "Orgn" in tags:
        org = word
      
    
  greets.append(greet)
  byes.append(bye)
  introduced.append(intro)
  names.append(name)
  orgs.append(org)

In [64]:
for i in names:
  if i != 'None':
    print(i)

ангелина
ангелина
ангелина
максим
анастасия


In [63]:
len(greets), len(byes), len(introduced), len(names), len(orgs)

(480, 480, 480, 480, 480)

In [71]:
df['greets'] = greets
df['introduced'] = introduced
df['manager_name'] = names
df['organization'] = orgs
df['goodbyes'] = byes

In [72]:
df.head()

,dlg_id,line_n,role,text,text_norm,greets,introduced,manager_name,organization,goodbyes
0,0,0,client,Алло,[алло],False,False,None,None,False
1,0,1,manager,Алло здравствуйте,"[алло, здравствуйте]",True,False,None,None,False
2,0,2,client,Добрый день,"[добрый, день]",False,False,None,None,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,"[я, звать, ангелина, компания, диджитал, бизне...",False,True,ангелина,None,False
4,0,4,client,Ага,[ага],False,False,None,None,False


In [94]:
def greet_bye_check(df, i):
  greet_res = df['greets'].value_counts()
  #print(greet_res)
  bye_res = df['goodbyes'].value_counts()
  #print(bye_res)
  if greet_res.size == 2 and bye_res.size == 2:
    print(f'Both greeting and goodbye are present in session {i}')
  #print('\n')

for i in range(6):
  df_small = df[df['dlg_id'] == i]
  greet_bye_check(df_small, i)

Both greeting and goodbye are present in session 0
Both greeting and goodbye are present in session 1
